In [1]:
import numpy
import numpy as np
import sys
sys.path.append('Models')
import Ionization.Ionization as Ionization
import Chemistry.Chemistry as Chemistry
import datetime

import IRI2016
import matplotlib.pyplot as plt
import scipy.optimize
import tables
from scipy.interpolate import interp1d
import scipy.optimize as op
import emcee
import pickle
from tools.loggerinit.LoggerInit import *
from tools.configreader.ConfigReader import *
from scipy.optimize import fmin_l_bfgs_b
from scipy import optimize

In [ ]:
## This is all in python 2!

In [2]:
iz = Ionization.Ionization()
iri2016 = IRI2016.IRI2016()
chem = Chemistry.Chemistry(SteadyStateTime = 100.,ISRIntegrationTime = 100.)

In [6]:
# read in the data
inFile = '/Users/srkaeppler/Dropbox/research/data/RileyRuns/troyer-sk2005-inversion/20161013.003_bc_nenotr_1min.h5'
inData = tables.open_file(inFile)
Ne = inData.root.NeFromPower.Ne_NoTr.read()/1e6
Alt = inData.root.NeFromPower.Altitude.read()/1000.
dNe = inData.root.NeFromPower.errNe_NoTr.read()
dNe = numpy.sqrt(dNe)/1e6
unixTime = numpy.mean(inData.root.Time.UnixTime.read(), axis=1)
print 'dNe', dNe.shape

glat = inData.root.Site.Latitude.read()
glon = inData.root.Site.Longitude.read()
inData.close()

dNe (210, 4, 140)


In [15]:

# which beam and which time index
BeamNoIn = 1
IndexIn = 151

BeamNumber = int(BeamNoIn)
TimeIndex = int(IndexIn)

# need to automate the time slice
tmpNe = Ne[TimeIndex,BeamNumber,:]
tmpAlt = Alt[BeamNumber,:]
tmpdNe = dNe[TimeIndex,BeamNumber,:]

# filter the data
        # these should be set by config files
qcutoff = numpy.where( (tmpAlt >  69.) \
                        & (tmpAlt < 141.))

tmpY = tmpAlt[qcutoff]
tmpZ = tmpNe[qcutoff]
tmpdZ = tmpdNe[qcutoff]

In [16]:
# setting altitude cut offs
# these are set by config files
AltMinKm = 70.
AltMaxKm = 140.
AltStepKm = 1.

# create interpolated altitude grid, Riley did the same thing.
# this is a little bit different than what Riley did.
Y = numpy.arange(AltMinKm,AltMaxKm,AltStepKm)
fZ = interp1d(tmpY,tmpZ)
fdZ = interp1d(tmpY,tmpdZ)
Z = fZ(Y)
dZ = fdZ(Y)*Z
tUnix = unixTime[TimeIndex]
# print 'Y, dZ', Y, dZ


In [17]:
# create the energy grid:
# this is exactly the same as Riley
EeV = numpy.logspace(np.log10(1000), np.log10(500000), 50)

In [18]:
# OK so now run the initialization to get the background ionosphere in the D-region
chem.Set_Inital_Ionization(tUnix,glat,glon,AltMinKm,AltMaxKm,AltStepKm)

366
366
70
izMin [10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33
 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57
 58 59 60 61 62 63 64 65 66 67 68 69]
q0 (array([0]),)
i0 10
iz y0, 0 [0.02759406 0.00771425 0.13760271 0.11780812]
iz y0, 1 [0.04365796 0.01214825 0.15939912 0.12841708]
iz y0, 2 [0.07496504 0.02061225 0.20079647 0.14897666]
iz y0, 3 [0.13587214 0.03647242 0.26902064 0.17958398]
iz y0, 4 [0.25320136 0.06513976 0.3714863  0.21730473]
iz y0, 5 [0.47291566 0.11376588 0.51604664 0.25783521]
iz y0, 6 [0.87278115 0.19057497 0.70981165 0.29563144]
iz y0, 7 [1.58235473 0.30364086 0.95945303 0.3257267 ]
iz y0, 8 [2.81626446 0.46004554 1.27213878 0.34513975]
iz y0, 9 [4.92943107 0.66599317 1.65677631 0.35329585]
iz y0, 10 [8.50839367 0.92780693 2.12517205 0.35152027]
iz y0, 11 [29.82539538  2.57532928  5.51326415  0.70316474]
iz y0, 12 [76.2379146   5.13681946 10.37137987  1.01860028]
iz y0, 13 [148.99992691   7.74914491  14.83361336 

In [19]:
# ok so now we can basically run the initialization script which converts the electron density to ionization rate.
# note that this is in CGS units and Riley did everything in SI so some conversions might be needed.
# remember I designed that you need to run the 'init' function above anytime you change location or altitude.
# so currently the Chem class is 'baked in' for the time and location of interest.

Qz,NeOut2 = chem.Calculate_Ionization_From_Ne(Y,Z, chem.DregionChem)

Ne2QZ iz,indx, IRIiz 0 0 70.0 70.0 3284.732397367787 3284.7344144148965 793.548583984375
Ne2QZ iz,indx, IRIiz 1 1 71.0 71.0 2547.9227149483713 2547.92224581228 373.2406005859375
Ne2QZ iz,indx, IRIiz 2 2 72.0 72.0 2402.6113720214007 2402.610241215014 234.44110107421875
Ne2QZ iz,indx, IRIiz 3 3 73.0 73.0 9019.432592165324 9019.429689816025 1142.48388671875
Ne2QZ iz,indx, IRIiz 4 4 74.0 74.0 7944.15083618869 7944.152757201268 629.763916015625
Ne2QZ iz,indx, IRIiz 5 5 75.0 75.0 13063.896962250834 13063.897048943038 833.398681640625
Ne2QZ iz,indx, IRIiz 6 6 76.0 76.0 15775.928312277214 15775.93153562913 764.091552734375
Ne2QZ iz,indx, IRIiz 7 7 77.0 77.0 17696.863996654036 17696.859935355205 674.281982421875
Ne2QZ iz,indx, IRIiz 8 8 78.0 78.0 21953.19581755137 21953.19714035274 738.086181640625
Ne2QZ iz,indx, IRIiz 9 9 79.0 79.0 26983.980995103917 26983.97647281646 854.451416015625
Ne2QZ iz,indx, IRIiz 10 10 80.0 80.0 20937.669345838425 20937.672640631175 500.184326171875
Ne2QZ iz,indx, IRI

In [20]:
print 'Qz.shape, Z.shape', Qz.shape, Z.shape
print Qz

Qz.shape, Z.shape (70,) (70,)
[  793.54858398   373.24060059   234.44110107  1142.48388672
   629.76391602   833.39868164   764.09155273   674.28198242
   738.08618164   854.45141602   500.18432617  1144.14892578
  2639.04394531  3010.08300781  3132.93652344  5131.34179688
  6200.72851562  7857.67578125  8978.20703125  9685.59765625
 12707.02734375 12222.95703125 13263.36328125 15044.05859375
 14218.48828125 15422.04296875 16446.7890625  19560.9296875
 21895.2890625  21713.6953125  24444.2890625  27403.0234375
 22199.6328125  21706.7578125  22501.2265625  19495.3828125
 19515.8671875  17133.8984375  15078.28515625 15092.31640625
 14284.93359375 16068.4921875  15039.48046875 14853.23828125
 14787.75390625 11938.66796875  9800.26171875 12239.53515625
 11481.83203125 10940.32421875  6461.78710938  6833.94335938
  5709.27929688  6130.65039062  7119.08398438  7433.52929688
  7005.63867188  6209.25585938  6258.01757812  5813.90039062
  5714.22070312  4229.13476562  4570.56445312  3893.643554

In [21]:
# convert to SI
qz_estimate_SI = Qz*(100.**3.)
dq_estimate_SI = 0.15*qz_estimate_SI # make the uncertainty a uniform 15% uncertainty

In [22]:
qz_estimate_SI # looks like a slightly higher magnitude than Riley's inversions...

array([7.93548584e+08, 3.73240601e+08, 2.34441101e+08, 1.14248389e+09,
       6.29763916e+08, 8.33398682e+08, 7.64091553e+08, 6.74281982e+08,
       7.38086182e+08, 8.54451416e+08, 5.00184326e+08, 1.14414893e+09,
       2.63904395e+09, 3.01008301e+09, 3.13293652e+09, 5.13134180e+09,
       6.20072852e+09, 7.85767578e+09, 8.97820703e+09, 9.68559766e+09,
       1.27070273e+10, 1.22229570e+10, 1.32633633e+10, 1.50440586e+10,
       1.42184883e+10, 1.54220430e+10, 1.64467891e+10, 1.95609297e+10,
       2.18952891e+10, 2.17136953e+10, 2.44442891e+10, 2.74030234e+10,
       2.21996328e+10, 2.17067578e+10, 2.25012266e+10, 1.94953828e+10,
       1.95158672e+10, 1.71338984e+10, 1.50782852e+10, 1.50923164e+10,
       1.42849336e+10, 1.60684922e+10, 1.50394805e+10, 1.48532383e+10,
       1.47877539e+10, 1.19386680e+10, 9.80026172e+09, 1.22395352e+10,
       1.14818320e+10, 1.09403242e+10, 6.46178711e+09, 6.83394336e+09,
       5.70927930e+09, 6.13065039e+09, 7.11908398e+09, 7.43352930e+09,
      

In [ ]:
# so now comes the real test, if I generate my A matrix relative to Riley's, 